In [ ]:
TOKEN = "<YOUR TOKEN>"

In [2]:
from typing import List, Dict, Optional
from pydantic import BaseModel


class DataSource(BaseModel):
    name: str
    description: str
    data_schema: Dict[str, str] # column_name: type
    type: str  # 'table', 'csv', 'api', etc.
    database: str
    access_method: str | None = None
    limitations: Optional[str] | None = None
    recommendations: List[str] | None = None
    connection_params: Dict[str, str] | None = None

class Metric(BaseModel):
    name: str
    description: str
    calculation_method: str | None = None  # SQL or pseudo-code
    visualization_method: str | None = None # e.g. barchart, "гистограмма" и т.д.

class DWH(BaseModel):
    database: str
    structure: Dict[str, str] | None = None
    limitations: Optional[str] | None = None
    connection_params: Dict[str, str] | None = None

class BusinessProcess(BaseModel):
    name: str
    description: str
    schedule: str
    roles: List[Dict[str, str]] | None = None
    goals: List[str] | None = None
    limitations: Optional[str] | None = None

class Transformation(BaseModel):
    name: str
    logic: str # SQL or pseudo-code

class AnalyticsSpec(BaseModel):
    business_process: BusinessProcess
    data_sources: List[DataSource]
    metrics: List[Metric]
    dwh: DWH
    transformations: List[Transformation]

In [9]:
user_data = """
Мы хотим построить аналитику для интернет-магазина.
Основная цель — анализировать продажи и поведение покупателей, чтобы повысить выручку и оптимизировать маркетинговые кампании.
Пользователи аналитики — менеджеры по продажам и маркетологи.

Данные хранятся в PostgreSQL. Есть две основные таблицы:

orders: содержит информацию о заказах (order_id, product_id, timestamp, customer_id, amount)
customers: содержит информацию о клиентах (customer_id, name, region_id, age)
Ключевые метрики:

* Общая сумма продаж по дням
* Количество уникальных покупателей по регионам
* Средний чек

Данные должны обновляться ежедневно, желательно ночью в промежуток между 00:00 и 05:00.
В дальнейшем планируется добавить витрины для анализа повторных покупок и сегментации клиентов.
Важно учитывать возможные ограничения по GDPR.
"""

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser


llm = ChatOpenAI(
    model="just-ai/gigachat/GigaChat-2-Pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=TOKEN,
    base_url="https://caila.io/api/adapters/openai"
)


In [ ]:

system_template = """
Ты опытный аналитик данных, собирающий информацию от пользователя для дальнейшего построения аналитической системы 
(ETL/ETL-пайплайны, хранилище данных, дашборды).

Твоя задача - извлечь из пользовательского описания полезную информацию и структурировать её в следующем виде:
{{
  "business_process": {{
    "name": str,
    "description": str,
    "schedule": "",  # как часто надо обновлять данные
    "roles": [ {{"role": str}}, ... ], # кто будет использовать систему
    "goals": [str, ...], # какие цели хочется достичь
    "limitations": str | null # какие-то ограничения
  }},
  "data_sources": [
    {{
      "name": str,
      "description": str,
      "data_schema": {{ "column_name": "type", ... }},
      "type": str,
      "database": str,
      "access_method": str | null,
      "limitations": str | null,
      "recommendations": [str, ...] | null,
      "connection_params": {{ "param": "value", ... }} | null
    }},
    ...
  ],
  "metrics": [
    {{
      "name": str,
      "description": str,
      "calculation_method": str | null,
      "visualization_method": str | null
    }},
    ...
  ],
  "dwh": {{
    "database": str,
    "structure": {{ "table": "description", ... }} | null,
    "limitations": str | null,
    "connection_params": {{ "param": "value", ... }} | null
  }},
  "transformations": [
    {{
      "name": str,
      "logic": str
    }}
  ]
}}

Требования:
1) формат вывода - строго валидный JSON
2) если какой-то информации нет, оставь поле пустым (либо None)

"""
user_template = "{user_data}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

parser = JsonOutputParser()

chain = prompt_template | llm | parser


In [17]:

result = chain.invoke(
    {
        'user_data': user_data
    }
)

In [18]:
result

{'business_process': {'name': 'Анализ продаж и поведения покупателей интернет-магазина',
  'description': 'Анализировать продажи и поведение покупателей для повышения выручки и оптимизации маркетинговых кампаний',
  'schedule': 'ежедневно, ночью в промежуток между 00:00 и 05:00',
  'roles': [{'role': 'менеджеры по продажам'}, {'role': 'маркетологи'}],
  'goals': ['повышение выручки', 'оптимизация маркетинговых кампаний'],
  'limitations': 'ограничения по GDPR'},
 'data_sources': [{'name': 'orders',
   'description': 'информация о заказах',
   'data_schema': {'order_id': 'int',
    'product_id': 'int',
    'timestamp': 'timestamp',
    'customer_id': 'int',
    'amount': 'float'},
   'type': 'таблица',
   'database': 'PostgreSQL',
   'access_method': None,
   'limitations': None,
   'recommendations': None,
   'connection_params': None},
  {'name': 'customers',
   'description': 'информация о клиентах',
   'data_schema': {'customer_id': 'int',
    'name': 'string',
    'region_id': 'int

In [23]:
business_process = BusinessProcess(**result['business_process'])
data_sources = [DataSource(**ds) for ds in result['data_sources']]
metrics = [Metric(**m) for m in result['metrics']]
dwh = DWH(**result['dwh'])
transformations = [Transformation(**t) for t in result['transformations']]

spec = AnalyticsSpec(business_process=business_process,
                     data_sources=data_sources,
                     metrics=metrics,
                     dwh=dwh,
                     transformations=transformations)
spec

AnalyticsSpec(business_process=BusinessProcess(name='Анализ продаж и поведения покупателей интернет-магазина', description='Анализировать продажи и поведение покупателей для повышения выручки и оптимизации маркетинговых кампаний', schedule='ежедневно, ночью в промежуток между 00:00 и 05:00', roles=[{'role': 'менеджеры по продажам'}, {'role': 'маркетологи'}], goals=['повышение выручки', 'оптимизация маркетинговых кампаний'], limitations='ограничения по GDPR'), data_sources=[DataSource(name='orders', description='информация о заказах', data_schema={'order_id': 'int', 'product_id': 'int', 'timestamp': 'timestamp', 'customer_id': 'int', 'amount': 'float'}, type='таблица', database='PostgreSQL', access_method=None, limitations=None, recommendations=None, connection_params=None), DataSource(name='customers', description='информация о клиентах', data_schema={'customer_id': 'int', 'name': 'string', 'region_id': 'int', 'age': 'int'}, type='таблица', database='PostgreSQL', access_method=None, li

In [22]:
data_sources

[DataSource(name='orders', description='информация о заказах', data_schema={'order_id': 'int', 'product_id': 'int', 'timestamp': 'timestamp', 'customer_id': 'int', 'amount': 'float'}, type='таблица', database='PostgreSQL', access_method=None, limitations=None, recommendations=None, connection_params=None),
 DataSource(name='customers', description='информация о клиентах', data_schema={'customer_id': 'int', 'name': 'string', 'region_id': 'int', 'age': 'int'}, type='таблица', database='PostgreSQL', access_method=None, limitations=None, recommendations=None, connection_params=None)]